In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
hdf = pd.read_parquet('02-gig-economy/data/hhv1.parquet')
pdf = pd.read_parquet('02-gig-economy/data/perv1.parquet')
pdf = pdf.droplevel('visit')
hdf.head()

visit  \
yr_quarter sector FSU   sample-sg-sb-no second-stage-stratum-no sample-hh-no         
Q1         rural  10002 1               1                       01              V1   
                                        2                       01              V1   
                                                                02              V1   
                                        3                       01              V1   
                        2               1                       01              V1   

                                                                                         state  \
yr_quarter sector FSU   sample-sg-sb-no second-stage-stratum-no sample-hh-no                     
Q1         rural  10002 1               1                       01            Himachal Pradesh   
                                        2                       01            Himachal Pradesh   
                                                                02            Himachal Pradesh   
                                        3                       01            Himachal Pradesh   
                        2               1                       01            Himachal Pradesh   

                                                                             district  \
yr_quarter sector FSU   sample-sg-sb-no second-stage-stratum-no sample-hh-no            
Q1         rural  10002 1               1                       01                 04   
                                        2                       01                 04   
                                                                02                 04   
                                        3                       01                 04   
                        2               1                       01                 04   

                                                                             nss-region  \
yr_quarter sector FSU   sample-sg-sb-no second-stage-stratum-no sample-hh-no              
Q1         rural  10002 1               1                       01                  021   
                                        2                       01                  021   
                                                                02                  021   
                                        3                       01                  021   
                        2               1                       01                  021   

                                                                             stratum  \
yr_quarter sector FSU   sample-sg-sb-no second-stage-stratum-no sample-hh-no           
Q1         rural  10002 1               1                       01                01   
                                        2                       01                01   
                                                                02                01   
                                        3                       01                01   
                        2               1                       01                01   

                                                                             sub-stratum  \
yr_quarter sector FSU   sample-sg-sb-no second-stage-stratum-no sample-hh-no               
Q1         rural  10002 1               1                       01                    14   
                                        2                       01                    14   
                                                                02                    14   
                                        3                       01                    14   
                        2               1                       01                    14   

                                                                             sub-sample  \
yr_quarter sector FSU   sample-sg-sb-no second-stage-stratum-no sample-hh-no              
Q1         rural  10002 1               1                       01               

In [3]:
# Get MPCE info

mce = hdf['monthly_consumer_expenditure_others'] / hdf['hh_size']
hdf['mpce_qrt'] = pd.qcut(mce, 4, labels=[1, 2, 3, 4])
pdf['mpce_qrt'] = hdf['mpce_qrt'].reindex(pdf.index.droplevel('person_serial_number')).values

In [4]:
# Get NCO

NCO_COLS = ['usual_principal_activity_nco', 'usual_subsidiary_activity_nco', 'weekly_occupation_nco']
ORG_NCO_CODES = [214, 215, 2161, 241, 242, 243, 311, 321, 331, 332, 333, 343, 351, 352, 412, 413, 421, 422, 431, 432, 441, 511, 512, 513, 515, 521, 522, 524, 531, 532, 832, 833, 911, 912, 941, 952]
pdf['has_NCO'] = pdf[NCO_COLS].isin(list(map(str, ORG_NCO_CODES))).any(axis=1)

In [5]:
# Get NIC

NIC_COLS = ['usual_principal_activity_nic', 'usual_subsidiary_activity_nic', 'weekly_industry_nic']
ORG_NIC_CODES = list(map(str, [
    10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 
    30, 31, 32, 33, 35, 36, 37, 38, 39, 41, 42, 43, 47, 49, 50, 51, 52, 53, 55, 56, 
    58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 77, 78, 79, 80, 81, 82, 85, 87, 95
]))
pdf['has_NIC'] = False
for col in NIC_COLS:
    pdf['has_NIC'] |= pdf[col].str[:2].isin(ORG_NIC_CODES)

* Largely limiting to urban (study actually covers top 100 cities), non-agricultural workforce.
* 18-45 years of age
* Secondary school to graduation.
* Below the 75th percentile of MPCE

In [6]:
df = pdf[pdf.index.get_level_values(1) == 'urban']
df = df[df['age'] >= 18]
df = df[df['age'] <= 45]

In [8]:
df = df[df['general_education'] != 'postgraduate and above']
df = df[df['mpce_qrt'].isin([1, 2, 3])]
df = df[df['has_NIC'] & df['has_NCO']]

In [11]:
df.to_parquet('02-gig-economy/data/gig-economy.parquet')